# uv --link-mode=copy 検証

## 目的

`--link-mode=copy` オプションを使うことで、Databricks の `/Workspace` 上でも `uv run` が動作するか検証する。

## 背景

- `/Workspace` はネットワークファイルシステムであり、ハードリンクの作成に制限がある
- `uv run` はデフォルトでハードリンクを使用するため、`os error 18` (Invalid cross-device link) が発生する
- `--link-mode=copy` を使えばハードリンクの代わりにコピーを使用するため、この問題を回避できる可能性がある

## 1. 環境確認

In [ ]:
import os
import subprocess

cwd = os.getcwd()
print(f"CWD: {cwd}")
print(f"Python: {os.popen('which python').read().strip()}")
print(f"/Workspace 上か: {cwd.startswith('/Workspace')}")

pyproject_exists = os.path.exists(os.path.join(cwd, 'pyproject.toml'))
print(f"pyproject.toml exists: {pyproject_exists}")

PROJECT_DIR = cwd if pyproject_exists else None

In [ ]:
# ファイルシステム情報
print("=== ファイルシステム情報 ===")
!df -h /Workspace 2>/dev/null || echo "/Workspace の df 取得失敗"
print()
!df -h /usr/bin 2>/dev/null || echo "/usr/bin の df 取得失敗"

## 2. uv インストール

In [ ]:
%pip install uv -q

In [ ]:
!uv --version

## 3. uv sync (--link-mode=copy なし)

In [ ]:
import shutil

# .venv が存在すれば削除
venv_path = os.path.join(PROJECT_DIR, ".venv") if PROJECT_DIR else None
if venv_path and os.path.exists(venv_path):
    shutil.rmtree(venv_path)
    print(f"Removed {venv_path}")

In [ ]:
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "sync", "--no-dev"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv sync --no-dev ===")
    print(result.stderr)
    print(f"Return code: {result.returncode}")
else:
    print("PROJECT_DIR が設定されていません")

In [ ]:
# .venv の構造を確認
print("=== .venv/bin/ の内容 ===")
!ls -la .venv/bin/ 2>/dev/null | head -20

print("\n=== python3 シンボリックリンクの確認 ===")
!ls -la .venv/bin/python3 2>/dev/null
!readlink -f .venv/bin/python3 2>/dev/null || echo "readlink 失敗"

print("\n=== site-packages の内容 ===")
!ls .venv/lib/python*/site-packages/ 2>/dev/null | head -20

In [ ]:
# uv run を試す
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "run", "--no-dev", "python", "-c", "import httpx; print(f'httpx version: {httpx.__version__}')"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv run (--link-mode=copy なし) ===")
    if result.stdout:
        print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    print(f"Return code: {result.returncode}")

## 4. uv sync --link-mode=copy

In [ ]:
# .venv 削除
if venv_path and os.path.exists(venv_path):
    shutil.rmtree(venv_path)
    print(f"Removed {venv_path}")

In [ ]:
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "sync", "--no-dev", "--link-mode=copy"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv sync --no-dev --link-mode=copy ===")
    print(result.stderr)
    print(f"Return code: {result.returncode}")

In [ ]:
# .venv の構造を確認
print("=== .venv/bin/ の内容 ===")
!ls -la .venv/bin/ 2>/dev/null | head -20

print("\n=== python3 シンボリックリンクの確認 ===")
!ls -la .venv/bin/python3 2>/dev/null
!readlink -f .venv/bin/python3 2>/dev/null || echo "readlink 失敗"

print("\n=== site-packages の内容 ===")
!ls .venv/lib/python*/site-packages/ 2>/dev/null | head -20

In [ ]:
# uv run を試す
if PROJECT_DIR:
    result = subprocess.run(
        ["uv", "run", "--no-dev", "--link-mode=copy", "python", "-c", "import httpx; print(f'httpx version: {httpx.__version__}')"],
        cwd=PROJECT_DIR,
        capture_output=True,
        text=True
    )
    print("=== uv run --link-mode=copy ===")
    if result.stdout:
        print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    print(f"Return code: {result.returncode}")

## 5. 結論

### 検証結果

| コマンド | 結果 |
|----------|------|
| `uv sync` | 成功 (警告付き: hardlink → copy にフォールバック) |
| `uv sync --link-mode=copy` | 成功 (警告なし) |
| `uv run` | 失敗 (`os error 18`) |
| `uv run --link-mode=copy` | 失敗 (`os error 18`) |

### 原因

`--link-mode=copy` はパッケージのインストール時のリンクモードを制御するが、`uv run` で発生するエラーは `.venv/bin/python3` のシンボリックリンクを正規化 (canonicalize) する処理で発生している。

```
/Workspace/.../project/        (NFS)
└── .venv/
    ├── bin/
    │   └── python3 → /usr/bin/python3  (シンボリックリンク → ローカル)
    └── lib/python3.12/site-packages/   (NFS)
        └── httpx/ 等
```

NFS 上のシンボリックリンクからローカルファイルシステムへの正規化で `os error 18` が発生するため、`--link-mode=copy` では解決できない。